##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/building_your_own_federated_learning_algorithm"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Antes de começar

Antes de começar, execute o código abaixo para que o ambiente seja configurado corretamente. Se não for exibida uma saudação, consulte as instruções de [instalação](../install.md). 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

**OBSERVAÇÃO**: foi verificado que este Colab funciona com a [versão mais recente lançada](https://github.com/tensorflow/federated#compatibility) do pacote pip `tensorflow_federated`, mas o projeto Tensorflow Federated ainda está em desenvolvimento pré-lançamento e pode não funcionar no `main`.

# Crie seu próprio algoritmo de aprendizado federado

Nos tutoriais de [classificação de imagens](federated_learning_for_image_classification.ipynb) e [geração de texto](federated_learning_for_text_generation.ipynb), você aprendeu a configurar os pipelines de modelos e dados para o Aprendizado Federado (FL) e realizou o treinamento federado pela camada de API `tff.learning` do TFF.

Na pesquisa de FL, essa é só a ponta do iceberg. Este tutorial discute como implementar algoritmos de aprendizado federado *sem* usar a API `tff.learning`. Neste tutorial, você fará o seguinte:

**Objetivos:**

- Entender a estrutura geral dos algoritmos de aprendizado federado.
- Explorar o *Federated Core* do TFF.
- Usar o Federated Core para implementar diretamente o cálculo federado de médias.

Apesar deste tutorial ser completo, talvez seja útil conferir primeiro os tutoriais de [classificação de imagens](federated_learning_for_image_classification.ipynb) e [geração de texto](federated_learning_for_text_generation.ipynb).


## Prepare os dados de entrada

Primeiro, carregue e pré-processe o dataset EMNIST incluído no TFF. Para mais detalhes, veja o tutorial de [classificação de imagens](federated_learning_for_image_classification.ipynb).

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

Para alimentar o dataset no modelo, os dados são achatados e cada exemplo é convertido em uma tupla de formato `(flattened_image_vector, label)`.

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

Agora, selecione um número pequeno de clientes e aplique o pré-processamento acima aos datasets deles.

In [ ]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## Prepare o modelo

É usado o mesmo modelo do tutorial de [classificação de imagens](federated_learning_for_image_classification.ipynb). Esse modelo (implementado por `tf.keras`) tem uma única camada oculta, seguida por uma camada softmax.

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

Para usar esse modelo no TFF, envolva o modelo do Keras como um [`tff.learning.models.VariableModel`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model). Isso permite realizar o [passo para frente](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass) do modelo no TFF e [extrair saídas do modelo](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics). Para mais detalhes, veja também o tutorial de [classificação de imagens](federated_learning_for_image_classification.ipynb).

In [ ]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

Enquanto o `tf.keras` é usado acima para criar um `tff.learning.models.VariableModel`, o TFF oferece suporte a modelos mais gerais. Esses modelos têm os seguintes atributos relevantes que capturam os pesos dos modelos:

- `trainable_variables`: um iterável de tensores que correspondem a camadas treináveis.
- `non_trainable_variables`: um iterável dos tensores que correspondem a camadas não treináveis.

Neste tutorial, somente `trainable_variables` será usado (já que o modelo só tem esse).

# Crie seu próprio algoritmo de aprendizado federado

Embora a API `tff.learning` permita criar diversas variantes do cálculo federado de médias, há outros algoritmos federados que não se encaixam perfeitamente nesse framework. Por exemplo, talvez você queira acrescentar regularização, recorte ou algoritmos mais complicados como [treinamento federado de GAN](https://github.com/tensorflow/federated/tree/main/tensorflow_federated/python/research/gans). Talvez você também tenha interesse em [análise federada](https://ai.googleblog.com/2020/05/federated-analytics-collaborative-data.html).

Para esses algoritmos mais avançados, você terá que escrever seu próprio algoritmo personalizando usando o TFF. Em vários casos, os algoritmos federados têm 4 componentes principais:

1. Um passo de transmissão do servidor para o cliente.
2. Um passo de atualização do cliente local.
3. Um passo de upload do cliente para o servidor.
4. Um passo de atualização do servidor.

No TFF, um algoritmo federado é geralmente representado como um [`tff.templates.IterativeProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess) (que será chamado apenas de `IterativeProcess`). Essa é uma classe que contém funções `initialize` e `next`. Aqui, `initialize` é usada para inicializar o servidor, e `next` faz uma rodada de comunicação do algoritmo federado. Vamos escrever um esqueleto do nosso processo iterativo para o cálculo federado de médias.

Primeiro, há uma função de inicialização que simplesmente cria um `tff.learning.models.VariableModel` e retorna seus pesos treináveis.

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

Essa função parece boa, mas, como você verá depois, é necessário fazer uma pequena modificação para transformá-la em uma "computação do TFF".

Em seguida, vamos escrever um esboço da `next_fn`.

In [ ]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

Vamos focar na implementação desses quatro componentes separadamente. Primeiro, vamos nos concentrar nas partes que podem ser implementadas totalmente no TensorFlow, ou seja, os passos de atualização do cliente e do servidor.


## Blocos do TensorFlow 

### Atualização do cliente

O `tff.learning.models.VariableModel` pode ser usado para fazer o treinamento do cliente basicamente da mesma forma que o treinamento de um modelo do TensorFlow. Especificamente, é possível usar uma `tf.GradientTape` para computar o gradiente em lotes de dados e aplicar esse gradiente usando um `client_optimizer`. Isso só envolve os pesos treináveis.


In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

### Atualização do servidor

A atualização do servidor para o cálculo federado de médias é mais simples do que a atualização do cliente. Este tutorial implementa o cálculo federado de médias comum, em que os pesos do modelo do servidor são substituídos pela média dos pesos do modelo do cliente. Novamente, só são usados os pesos treináveis.

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

O fragmento pode ser simplificado ao apenas retornar `mean_client_weights`. No entanto, implementações mais avançadas do cálculo federado de médias usam `mean_client_weights` com técnicas mais sofisticadas, como momento ou adaptabilidade.

**Desafio**: implementar uma versão de `server_update` que atualiza os pesos do servidor para o ponto médio de model_weights e mean_client_weights. (Observação: essa abordagem de "ponto médio" é análoga ao trabalho recente sobre o [otimizador Lookahead](https://arxiv.org/abs/1907.08610)!).

Até agora, só foi utilizado código TensorFlow. Isso é intencional, já que o TTF permite usar grande parte do código TensorFlow com que você já está familiarizado. Em seguida, é preciso especificar a **lógica de orquestração**, ou seja, a lógica que dita o que o servidor transmite ao cliente e o que o cliente carrega no servidor.

Isso requer o *Federated Core* do TFF.

# Introdução ao Federated Core

O Federated Core (FC) é um conjunto de interfaces de nível inferior que serve como a base da API `tff.learning`. Porém, essas interfaces não estão limitadas ao aprendizado. De fato, podem ser usadas para análises e muitas outras computações de dados distribuídos.

Em um alto nível, o federated core é um ambiente de desenvolvimento que permite uma lógica de programa expressa de maneira compacta para combinar o código TensorFlow com operadores de comunicação distribuída (como somas e transmissões distribuídas). O objetivo é fornecer aos pesquisadores e usuários controle explícito da comunicação distribuída em seus sistemas, sem exigir detalhes da implementação do sistema (como especificar trocas de mensagens da rede ponto a ponto).

Um ponto fundamental é que o TFF foi criado para preservar a privacidade. Portanto, ele permite o controle explícito de onde os dados ficam, para evitar o acúmulo indesejado de dados no servidor centralizado.

## Dados federados

Um conceito importante no TFF são os "dados federados", que são uma coleção de itens de dados hospedados em grupo de dispositivos de um sistema distribuído (por exemplo, datasets do cliente ou pesos do modelo do servidor). A coleção inteira de valores em todos os dispositivos é representada como um único *valor federado*.

Por exemplo, suponha que cada dispositivo do cliente tenha um float representando a temperatura de um sensor. Esses floats podem ser representados como um *float federado* assim:

In [ ]:
federated_float_on_clients = tff.FederatedType(tf.float32, tff.CLIENTS)

Os tipos federados são especificados por um tipo `T` dos seus membros constituintes (por exemplo, `tf.float32`) e um grupo `G` de dispositivos. Geralmente, `G` é `tff.CLIENTS` ou `tff.SERVER`. Um tipo federado assim é representado como `{T}@G`, conforme exibido abaixo.

In [ ]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

Por que as colocações são tão importantes para o TFF? Um objetivo principal do TFF é permitir a escrita de código que possa ser implementado em um sistema distribuído real. Por isso, é essencial pensar sobre quais subconjuntos de dispositivos executam qual código e onde os diferentes conjuntos de dados ficam.

O TFF foca em três aspectos: *dados*, onde os dados são *colocados* e como os dados são *transformados*. Os dois primeiros são encapsulados em tipos federados, enquanto o último é encapsulado em *computações federadas*.

## Computações federadas

O TFF é um ambiente de programação funcional fortemente tipado, cujas unidades básicas são *computações federadas*. Elas são partes de lógica que recebem valores federados como entrada e retornam valores federados como saída.

Por exemplo, suponha que você queira fazer a média das temperaturas dos sensores do cliente. Você pode definir o seguinte (usando nosso float federado):

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def get_average_temperature(client_temperatures):
  return tff.federated_mean(client_temperatures)

Talvez você pergunte: como isso é diferente do decorador `tf.function` no TensorFlow? A resposta fundamental é que o código gerado por `tff.federated_computation` não é código TensorFlow nem Python, é uma especificação de um sistema distribuído em uma *linguagem glue* interna independente de plataforma.

Embora isso possa parecer complicado, pense nas computações do TFF como funções com assinaturas de tipo bem definidas. Essas assinaturas de tipo podem ser consultadas diretamente.

In [ ]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

`tff.federated_computation` recebe argumentos do tipo federado `<float>@CLIENTS` e retorna valores do tipo federado `<float>@SERVER`. As computações federadas também podem ir de servidor a cliente, de cliente a cliente e de servidor a servidor. As computações federadas também podem ser compostas como funções normais, desde que as assinaturas de tipo coincidam.

Para dar suporte ao desenvolvimento, o TFF permite invocar uma `tff.federated_computation` como uma função Python. Por exemplo, você pode chamar:

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

## Computações não eager e o TensorFlow

É preciso ter em mente duas restrições importantes. Primeiro, quando o interpretador Python encontra um decorador `tff.federated_computation`, é feito um único tracing da função e a serialização dela para uso futuro. Devido à natureza descentralizada do aprendizado federado, esse uso futuro pode ocorrer em outro lugar, como um ambiente de execução remota. Portanto, as computações do TFF são fundamentalmente *não eager*. Esse comportamento é de certa forma análogo ao do decorador [`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) no TensorFlow.

Segundo, uma computação federada pode consistir somente de operadores federados (como `tff.federated_mean`), que não podem conter operações do TensorFlow. O código TensorFlow precisa estar confinado aos blocos decorados com `tff.tf_computation`. A maior parte do código comum do TensorFlow pode ser decorado diretamente, como a seguinte função que recebe um número e adiciona `0.5` a ele.

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

Também há assinaturas de tipo, mas *sem colocações*. Por exemplo, você pode chamar:

In [ ]:
str(add_half.type_signature)

'(float32 -> float32)'

Isso mostra uma diferença importante entre `tff.federated_computation` e `tff.tf_computation`. O primeiro tem colocações explícitas, enquanto o segundo, não.

Você pode usar blocos de `tff.tf_computation` nas computações federadas com a especificação de colocações. Vamos criar uma função que adicione a metade, mas somente a floats federados nos clientes. Você pode fazer isso usando `tff.federated_map`, que aplica uma determinada `tff.tf_computation`, preservando a colocação.

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

Essa função é quase idêntica a `add_half`, exceto que recebe somente valores com colocação em `tff.CLIENTS` e retorna valores com a mesma colocação. É possível ver isso em sua assinatura de tipo:

In [ ]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

Resumindo:

- O TFF faz operações em valores federados.
- Cada valor federado tem um *tipo federado*, com um *tipo* (por exemplo, `tf.float32`) e uma *colocação* (por exemplo, `tff.CLIENTS`).
- Os valores federados podem ser transformados usando-se *computações federadas*, que devem ser decoradas com `tff.federated_computation`, e uma assinatura de tipo federado.
- O código TensorFlow deve ficar contido em blocos com decoradores `tff.tf_computation`.
- Esses blocos podem ser incorporados às computações federadas.


# Crie seu próprio algoritmo de aprendizado federado: revisitado

Agora que você teve uma breve visão do Federated Core, pode criar seu próprio algoritmo de aprendizado federado. Lembre-se de que definimos acima funções `initialize_fn` e `next_fn` para o algoritmo. A `next_fn` utiliza `client_update` e `server_update` que definimos usando código puro do TensorFlow.

Porém, para fazer do algoritmo uma computação federada, ambas as funções `next_fn` e `initialize_fn` precisam ser uma `tff.federated_computation`.

## Blocos do TensorFlow Federated 

### Crie a computação de inicialização

A função de inicialização será bem simples: você criará um modelo usando `model_fn`. Porém, lembre-se de que é preciso separar o código TensorFlow usando `tff.tf_computation`.

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

Em seguida, você pode passá-la diretamente para uma computação federada usando `tff.federated_value`.

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

### Crie a `next_fn`

Agora é possível usar o código de atualização do cliente e do servidor para escrever o algoritmo em si. Primeiro, transforme `client_update` em uma `tff.tf_computation` que receba datasets do cliente e pesos do servidor e gere como saída um tensor de pesos do cliente atualizados.

Você precisará dos tipos correspondentes para decorar corretamente a função. Felizmente, o tipo dos pesos do servidor pode ser extraído diretamente do modelo.

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

Vamos conferir a assinatura de tipo do dataset. Lembre-se de que você pegou imagens 28 x 28 (com rótulos inteiros) e as achatou.

In [ ]:
str(tf_dataset_type)

'<float32[?,784],int32[?,1]>*'

Também é possível extrair o tipo de pesos do modelo usando a função `server_init` acima.

In [ ]:
model_weights_type = server_init.type_signature.result

Ao avaliar a assinatura de tipos, você poderá ver a arquitetura do modelo.

In [ ]:
str(model_weights_type)

'<float32[784,10],float32[10]>'

Agora, crie a `tff.tf_computation` para a atualização do cliente.

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

A versão de `tff.tf_computation` da atualização do servidor pode ser definida de forma similar, usando os tipos que você já extraiu.

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

Por último, mas não menos importante, você precisa criar a `tff.federated_computation` que junta tudo isso. Essa função receberá dois *valores federados*, um correspondente aos pesos do servidor (com a colocação `tff.SERVER`) e outro correspondente aos datasets do cliente (com a colocação `tff.CLIENTS`).

Esses dois tipos foram definidos acima. Você só precisa definir a colocação correta para eles usando `tff.FederatedType`.

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

Lembra-se dos quatro elementos de um algoritmo de aprendizado federado?

1. Um passo de transmissão do servidor para o cliente.
2. Um passo de atualização do cliente local.
3. Um passo de upload do cliente para o servidor.
4. Um passo de atualização do servidor.

Agora que você construiu tudo isso, cada parte pode ser representada compactamente como uma única linha de código do TFF. É por essa simplicidade que você tomou cuidado redobrado ao especificar os tipos federados.

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

Agora você tem uma `tff.federated_computation` para o algoritmo de inicialização e para a execução de um passo do algoritmo. Para finalizar o algoritmo, passe-as para `tff.templates.IterativeProcess`.

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

Vamos conferir a *assinatura de tipo* das funções `initialize` e `next` do processo iterativo.

In [ ]:
str(federated_algorithm.initialize.type_signature)

'( -> <float32[784,10],float32[10]>@SERVER)'

Isso reflete o fato de `federated_algorithm.initialize` ser uma função sem argumentos que retorna um modelo de uma única camada (com uma matriz de pesos 784 por 10 e 10 unidades de bias).

In [ ]:
str(federated_algorithm.next.type_signature)

'(<server_weights=<float32[784,10],float32[10]>@SERVER,federated_dataset={<float32[?,784],int32[?,1]>*}@CLIENTS> -> <float32[784,10],float32[10]>@SERVER)'

Aqui, é possível ver que `federated_algorithm.next` recebe um modelo do servidor e dados do cliente e retorna um modelo do servidor atualizado.

## Avalie o algoritmo

Vamos executar algumas rodadas e ver como a perda muda. Primeiro, defina uma função de avaliação usando a estratégia *centralizada* discutida no segundo tutorial.

Primeiro, crie um dataset de avaliação centralizado e depois aplique o mesmo pré-processamento usado para os dados de treinamento.

In [ ]:
central_emnist_test = emnist_test.create_tf_dataset_from_all_clients()
central_emnist_test = preprocess(central_emnist_test)

Em seguida, escreva uma função que receba um estado do servidor e use o Keras para avaliar com o dataset de teste. Se você já conhece o `tf.Keras`, reconhecerá este código, exceto pelo uso de `set_weights`.

In [ ]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

Agora, vamos inicializar o algoritmo e avaliar com o dataset de teste.

In [ ]:
server_state = federated_algorithm.initialize()
evaluate(server_state)

2042/2042 [==============================] - 2s 767us/step - loss: 2.8479 - sparse_categorical_accuracy: 0.1027


Vamos treinar por algumas rodadas e ver se algo muda.

In [ ]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

2042/2042 [==============================] - 2s 738us/step - loss: 2.5867 - sparse_categorical_accuracy: 0.0980


Há uma pequena diminuição na função de perda. Embora a mudança seja pequena, você só realizou 15 rodadas de treinamento com um subconjunto de clientes pequeno. Para obter resultados melhores, talvez seja necessário fazer centenas ou milhares de rodadas.

## Modifique o algoritmo

Agora, vamos parar e pensar no que foi feito. Você implementou o cálculo federado de médias diretamente pela combinação de código puro TensorFlow (para as atualizações do servidor e do cliente) com computações federadas usando o Federated Core do TFF.

Para fazer um aprendizado mais sofisticado, basta alterar o que está acima. Especificamente, ao editar o código puro TF acima, é possível mudar como o cliente faz o treinamento ou como o servidor atualiza seu modelo.

**Desafio:** acrescente o [recorte de gradiente](https://towardsdatascience.com/what-is-gradient-clipping-b8e815cdfb48) à função `client_update`.


Para alterações maiores, é possível fazer com que o servidor armazene e transmita mais dados. Por exemplo, o servidor pode armazenar a taxa de aprendizado do cliente e fazer com que ela decaia com o passar do tempo. No entanto, isso exige mudanças nas assinaturas de tipo usadas nas chamadas de `tff.tf_computation` acima.

**Desafio mais difícil:** implemente o cálculo federado de médias com o decaimento da taxa de aprendizado nos clientes.

Aqui, talvez você comece a perceber a flexibilidade em relação ao que é possível implementar nesse framework. Para ideias (incluindo a resposta do desafio mais difícil acima), veja o código-fonte de [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) ou confira os diversos [projetos de pesquisa](https://github.com/google-research/federated) que usam o TFF.